In [ ]:
pip install llmcompressor

In [2]:
pip list | grep compress

compressed-tensors        0.10.2
llmcompressor             0.6.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B"
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

In [15]:
from datasets import load_dataset

NUM_CALIBRATION_SAMPLES=512
MAX_SEQUENCE_LENGTH=2048

# Load dataset
ds = load_dataset("HuggingFaceH4/ultrachat_200k", split=f"train_sft[:{NUM_CALIBRATION_SAMPLES}]")
ds = ds.shuffle(seed=42)

# Preprocess the data into the format the model is trained with
def preprocess(example):
    return {"text": tokenizer.apply_chat_template(example["messages"], tokenize=False,)}
ds = ds.map(preprocess)

print("Columns in the dataset after pro-processing:" + ds.column_names)

top_1_row = ds.select(range(1))
for row in top_1_row:
    print("Top row data for the dataset after pro-processing:" + row)

# Tokenize the data (be careful with bos tokens - we need add_special_tokens=False since the chat_template already added it)
def tokenize(sample):
    return tokenizer(sample["text"], padding=False, max_length=MAX_SEQUENCE_LENGTH, truncation=True, add_special_tokens=False)
ds = ds.map(tokenize, remove_columns=ds.column_names)

print("Columns in the dataset after tokenizing and removing default dataset columns:"ds.column_names)

['prompt', 'prompt_id', 'messages', 'text']
['input_ids', 'attention_mask']


In [ ]:
from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier

# Configure the quantization algorithm to run
recipe = GPTQModifier(targets="Linear", scheme="W4A16", ignore=["lm_head"])

# Apply quantization
oneshot(
    model=model, dataset=ds,
    recipe=recipe,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
)

# Save to disk compressed
SAVE_DIR = MODEL_ID.rstrip("/").split("/")[-1] + "-W4A16-G128"
model.save_pretrained(SAVE_DIR, save_compressed=True)
tokenizer.save_pretrained(SAVE_DIR)

In [ ]:
pip install huggingface_hub

In [5]:
from huggingface_hub import login

login(token="<<your-token>>") # Replace with your actual token

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
repo_id = "balakriz/DeepSeek-R1-0528-Qwen3-8B-W4A16-G128" # Replace with your desired repo ID
api.create_repo(repo_id, exist_ok=True) # Create the repository if it doesn't exist

local_model_path = "./DeepSeek-R1-0528-Qwen3-8B-W4A16-G128"

# Push the model and tokenizer to the repository
api.upload_folder(
    folder_path=local_model_path,
    repo_id=repo_id,
    repo_type="model",
)